In [14]:
import os
import cv2

In [15]:
# 训练集转换
originLabelsDir = './labels/train/'
# 转换后的文件保存路径
saveDir = 'train-coco.txt'                                                                         
# 原始标签对应的图片路径
originImagesDir = './images/train/' 

In [2]:
# 验证集转换
originLabelsDir = './labels/val/'
# 转换后的文件保存路径
saveDir = 'val-coco.txt'                                                                         
# 原始标签对应的图片路径
originImagesDir = './images/val/' 

In [ ]:
txtFileList = os.listdir(originLabelsDir)
with open(saveDir, 'w') as fw:
    for txtFile in txtFileList:
        with open(os.path.join(originLabelsDir, txtFile), 'r') as fr:
            labelList = fr.readlines()
            for label in labelList:
                label = label.strip().split()
                x = float(label[1])
                y = float(label[2])
                w = float(label[3])
                h = float(label[4])
 
                # convert x,y,w,h to x1,y1,x2,y2
                imagePath = os.path.join(originImagesDir,
                                         txtFile.replace('txt', 'jpg'))
                image = cv2.imread(imagePath)
                H, W, _ = image.shape
                x1 = (x - w / 2) * W
                y1 = (y - h / 2) * H
                x2 = (x + w / 2) * W
                y2 = (y + h / 2) * H
                # 为了与coco标签方式对，标签序号从1开始计算
                fw.write(txtFile.replace('txt', 'jpg') + ' {} {} {} {} {}\n'.format(int(label[0]) + 1, x1, y1, x2, y2))
 
        print('{} done'.format(txtFile))

# 从这里开始 一个dataset字典

In [5]:
import json
import os
import cv2
 
# ------------用os提取images文件夹中的图片名称，并且将BBox都读进去------------
# 根路径，里面包含images(图片文件夹)，annos.txt(bbox标注)，classes.txt(类别标签),
# 以及annotations文件夹(如果没有则会自动创建，用于保存最后的json)
root_path = './'
# 用于创建训练集或验证集

 
# dataset用于保存所有数据的图片信息和标注信息
dataset = {'categories': [], 'annotations': [], 'images': []}
 
# 打开类别标签
with open(os.path.join(root_path, 'classes.txt')) as f:
    classes = f.read().strip().split()

In [6]:
 
#建立类别标签和数字id的对应关系

for i, cls in enumerate(classes, 1):
    dataset['categories'].append({'id': i, 'name': cls, 'supercategory': 'mark'})
 
# 读取images文件夹的图片名称
train_indexes = os.listdir(os.path.join(root_path, 'images/train/'))
val_indexes = os.listdir(os.path.join(root_path, 'images/val/'))

In [7]:
# 统计处理图片的数量
global count, anno_id
count = 0
anno_id =0  
 
# 读取Bbox信息
with open(os.path.join(root_path, 'train-coco.txt')) as tr:
    annos = tr.readlines()
 
    # ---------------接着将，以上数据转换为COCO所需要的格式---------------
    for k, index in enumerate(train_indexes):
        count += 1
        # 用opencv读取图片，得到图像的宽和高
        im = cv2.imread(os.path.join(root_path, 'images/train/') + index)
        height, width, _ = im.shape
 
        # 添加图像的信息到dataset中
        dataset['images'].append({'file_name': index,
                                  'id': k,
                                  'width': width,
                                  'height': height})
    #遍历注释anno
        for ai, anno in enumerate(annos):
            parts = anno.strip().split()
            # 如果图像的名称和标记的名称对上，则添加标记
            if parts[0] == index:
                # 类别
                cls_id = parts[1]
                # x_min
                x1 = float(parts[2])
                # y_min
                y1 = float(parts[3])
                # x_max
                x2 = float(parts[4])
                # y_max
                y2 = float(parts[5])
                width = max(0, x2 - x1)
                height = max(0, y2 - y1)
                anno_id+=1
                dataset['annotations'].append({
                    'area': width * height,
                    'bbox': [x1, y1, width, height],
                    'category_id': int(cls_id),
                    'id': anno_id,#每个注解一个id
                    'image_id': k,
                    'iscrowd': 0,
                    # mask, 矩形是从左上角点按顺时针的四个顶点
                    'segmentation': [[x1, y1, x2, y1, x2, y2, x1, y2]]
                })
                print('anno id :',anno_id)
    print('{} images handled'.format(count))
    

anno id : 1
anno id : 2
anno id : 3
anno id : 4
anno id : 5
anno id : 6
anno id : 7
anno id : 8
anno id : 9
anno id : 10
anno id : 11
anno id : 12
anno id : 13
anno id : 14
anno id : 15
anno id : 16
anno id : 17
anno id : 18
anno id : 19
anno id : 20
anno id : 21
anno id : 22
anno id : 23
anno id : 24
anno id : 25
anno id : 26
anno id : 27
anno id : 28
anno id : 29
anno id : 30
anno id : 31
anno id : 32
anno id : 33
anno id : 34
anno id : 35
anno id : 36
anno id : 37
anno id : 38
anno id : 39
anno id : 40
anno id : 41
anno id : 42
anno id : 43
anno id : 44
anno id : 45
anno id : 46
anno id : 47
anno id : 48
anno id : 49
anno id : 50
anno id : 51
anno id : 52
anno id : 53
anno id : 54
anno id : 55
anno id : 56
anno id : 57
anno id : 58
anno id : 59
anno id : 60
anno id : 61
anno id : 62
anno id : 63
anno id : 64
anno id : 65
anno id : 66
anno id : 67
anno id : 68
anno id : 69
anno id : 70
anno id : 71
anno id : 72
anno id : 73
anno id : 74
anno id : 75
anno id : 76
anno id : 77
anno id 

In [4]:
root_path = './'
#phase = 'val'    
phase ='train' # 需要修正
# 保存结果的文件夹
folder = os.path.join(root_path, 'annotations')
if not os.path.exists(folder):
  os.makedirs(folder)
json_name = os.path.join(root_path, 'annotations/instances_{}_images.json'.format(phase))
with open(json_name, 'w') as f:
  json.dump(dataset, f)